In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, Dropout
from keras import Sequential

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [2]:
model = ResNet50(weights="imagenet")

In [3]:
import numpy as np
from keras.utils import img_to_array, array_to_img, load_img

In [4]:
image_path = '/Users/pawankumarkc/Downloads/images.jpeg'
img = image.load_img(image_path, target_size=(224,224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [5]:
preds = model.predict(x)
print("Predicted: ", decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 401ms/step
Predicted:  [('n02124075', 'Egyptian_cat', 0.82269025), ('n02123045', 'tabby', 0.09589368), ('n02127052', 'lynx', 0.015513909)]


In [15]:
!mkdir -p ~/.Kaggle
!cp /Users/pawankumarkc/Downloads/kaggle.json ~/.Kaggle/

In [17]:
!kaggle datasets download -d salader/dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [18]:
#Generators dataset

train_ds = keras.utils.image_dataset_from_directory(
    directory='/Users/pawankumarkc/Documents/vscode_workspace/MLAlgo/CNN/dogs_vs_cats/train', 
    labels='inferred', 
    label_mode='int', 
    batch_size=32, 
    image_size=(256,256)
    )

val_ds = keras.utils.image_dataset_from_directory(
    directory='/Users/pawankumarkc/Documents/vscode_workspace/MLAlgo/CNN/dogs_vs_cats/test', 
    labels='inferred', 
    label_mode='int', 
    batch_size=32, 
    image_size=(256,256)
    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [19]:
#Normalization of image

def process(image, label):
    image = tf.cast(image/255., tf.float32)
    return image, label

In [20]:
train_ds = train_ds.map(process)

val_ds = val_ds.map(process)

In [21]:
model = Sequential()

#Block 1
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))

#Block 2
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))

#Block 3
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))

#Block 4
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))

#Block 5
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=2))

#Flatten
model.add(Flatten())

#Fully conneted layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))


#Fully conneted layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))


#Output = 1 as it is binary class (Cat or Dog)
model.add(Dense(1, activation='sigmoid'))

#Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      36928     
                                                                 
 batch_normalization (Batch  (None, 252, 252, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 122, 122, 128)     1

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)


Epoch 1/10
 56/625 [=>............................] - ETA: 1:41:11 - loss: 2.4741 - accuracy: 0.4983

KeyboardInterrupt: 

## Transfer learning model - VGG16

In [14]:
from keras.applications.vgg16 import VGG16


In [ ]:
conv_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

In [ ]:
conv_base.summary()

In [ ]:
model = Sequential()
model.add(conv_base) #CNN till here

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

#output
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, epochs=5, validation_data=val_ds)
